<a href="https://colab.research.google.com/github/bebechien/gemma/blob/main/Translator_of_Old_Korean_Literature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import os
from google.colab import userdata, drive

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
drive.mount('/content/drive')

# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp datasets
!pip install -q -U keras

os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
#os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

# Training Configurations
lora_name = "translator"
lora_rank = 8
lr_value = 1e-4

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Dataset

In [ ]:
import keras_nlp
tokenizer = keras_nlp.models.GemmaTokenizer.from_preset("gemma_instruct_2b_en")

from datasets import load_dataset
ds = load_dataset(
    "bebechien/HongGildongJeon",
    split="train",
)
print(ds)
data = ds.with_format("np", columns=["original", "modern translation"], output_all_columns=False)
train = []
max_token_len = 0

for x in data:
  item = f"<start_of_turn>user\n{x['original']}<end_of_turn>\n<start_of_turn>model\n{x['modern translation']}"
  train.append(item)
  length = tokenizer(item).get_shape().as_list()[0]
  if length > max_token_len:
    max_token_len = length
    print(f"longest toekn {max_token_len} - {item}")

print(train[0])
print(train[1])
print(train[2])

print(max_token_len)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['original', 'modern translation'],
    num_rows: 447
})
longest toekn 383 - <start_of_turn>user
됴션국셰둉ᄃᆡ왕즉위십오연의홍희문밧긔ᄒᆞᆫᄌᆡ상이잇스되셩은홍이요명은문이니위인이쳥염강직ᄒᆞ여덩망이거록ᄒᆞ니당셰의영웅이라일직용문의올나벼살이할림의쳐ᄒᆞ엿더니명망이됴졍의읏듬되ᄆᆡ젼하그덕망을승이녀긔ᄉᆞ벼살을도도와이조판셔로좌으졍을ᄒᆞ이시니승상이국은을감동ᄒᆞ야갈츙보국ᄒᆞ니ᄉᆞ방의일이업고도젹이업스ᄆᆡ시화연풍ᄒᆞ여나라이ᄐᆡ평ᄒᆞ더라<end_of_turn>
<start_of_turn>model
조선국 세종대왕 즉위 십오년에 홍회문 밖에 한 재상이 있으되, 성은 홍이요, 명은 문이니, 위인이 청렴강직하여 덕망이 거룩하니 당세의 영웅이라. 일찍 용문에 올라 벼슬이 한림에 처하였더니 명망이 조정의 으뜸 되매, 전하 그 덕망을 승히 여기사 벼슬을 돋우어 이조판서로 좌의정을 하게 하시니, 승상이 국은을 감동하여 갈충보국하니 사방에 일이 업고 도적이 없으매 시화연풍하여 나라가 태평하더라.
longest toekn 1173 - <start_of_turn>user
일일은승상난간의비겨잠ᄀᆞᆫ조의더니ᄒᆞᆫ풍이긜을인도ᄒᆞ여ᄒᆞᆫ고듸다다르니쳥산은암암ᄒᆞ고녹슈난양양ᄒᆞᆫ듸셰류쳔만ᄀᆞ지녹음이파ᄉᆞᄒᆞ고황금갓ᄐᆞᆫᄭᅬᄭᅩ리난춘흥을희롱ᄒᆞ여냥뉴간의왕ᄂᆡᄒᆞ며긔화요초만발ᄒᆞᆫᄃᆡ쳥학ᄇᆡᆨ학이며비취공작이춘광을ᄌᆞ랑ᄒᆞ거날승상이경물을귀경ᄒᆞ며졈졈드러가니만쟝졀벽은하날의다엇고구뷔구뷔벽계슈난골골이폭포되어오운이어러엿난ᄃᆡ길이ᄭᅳᆫ쳐갈바을모로더니문득쳥용이물결을혜치고머리을드러고함ᄒᆞ니산학이믄허지난듯ᄒᆞ더니그용이입을버리고긔운을토ᄒᆞ여승상의입으로드러뵈거날ᄭᆡ다르니평ᄉᆡᆼᄃᆡ몽이라ᄂᆡ염의혜아리되피련군ᄌᆞ을나희리라ᄒᆞ여즉시ᄂᆡ당의드러ᄀᆞ시비을믈이치고부인을익그러취침코져ᄒᆞ니부인이졍ᄉᆡᆨ왈승상은국지ᄌᆡ상이라쳬위존즁ᄒᆞ시거날ᄇᆡᆨ쥬의졍실의드러와노류장화갓치ᄒᆞ시니ᄌᆡ상의쳬면이어ᄃᆡ잇난잇ᄀᆞ승상이ᄉᆡᆼ각ᄒᆞ신직말ᄉᆞᆷ은당연ᄒᆞ

# Load Model

In [ ]:
import keras
import keras_nlp

import time

gemma = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
gemma.summary()

tick_start = 0

def tick():
    global tick_start
    tick_start = time.time()

def tock():
    print(f"TOTAL TIME ELAPSED: {time.time() - tick_start:.2f}s")

def text_gen(prompt):
    tick()
    input = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    output = gemma.generate(input, max_length=512)
    print("\nGemma output:")
    print(output)
    tock()

text_gen("ᄃᆡ작ᄒᆞ여그ᄭᅩᆺ치흣터지거ᄂᆞᆯ")
text_gen("금두겁이품의드러뵈니일졍ᄌᆡᄌᆞᄅᆞᆯ나흐리로다ᄒᆞ더니과연그ᄃᆞᆯ부터잉ᄐᆡᄒᆞ여십삭이차니")
text_gen("이ᄯᆡᄂᆞᆫᄉᆞ월초팔일이라이날밤의오ᄉᆡᆨ구룸이집을두루고향ᄂᆡ진동ᄒᆞ며션녀ᄒᆞᆫᄡᅣᆼ이촉을들고드러와김ᄉᆡᆼᄃᆞ려니르ᄃᆡ")
text_gen("ᄌᆡ히길너텬졍을어긔지말으소셔이아희ᄇᆡ필은낙양니샹셔집아ᄌᆡ니")


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)


Gemma output:
<start_of_turn>user
ᄃᆡ작ᄒᆞ여그ᄭᅩᆺ치흣터지거ᄂᆞᆯ<end_of_turn>
<start_of_turn>model
Sure, here's a translation of the phrase you provided:

"I am a person who is always searching for something."

This phrase is often used to describe someone who is curious and always looking for new information or experiences.
TOTAL TIME ELAPSED: 20.53s

Gemma output:
<start_of_turn>user
금두겁이품의드러뵈니일졍ᄌᆡᄌᆞᄅᆞᆯ나흐리로다ᄒᆞ더니과연그ᄃᆞᆯ부터잉ᄐᆡᄒᆞ여십삭이차니<end_of_turn>
<start_of_turn>model
금두겁이 품의드러뵈니일졍ᄌᆡᄌᆞᄅᆞᆯ나흐리로다ᄒᆞ더니과연그ᄃ misericordia에 대한 이야기입니다.
TOTAL TIME ELAPSED: 0.43s

Gemma output:
<start_of_turn>user
이ᄯᆡᄂᆞᆫᄉᆞ월초팔일이라이날밤의오ᄉᆡᆨ구룸이집을두루고향ᄂᆡ진동ᄒᆞ며션녀ᄒᆞᆫᄡᅣᆼ이촉을들고드러와김ᄉᆡᆼᄃᆞ려니르ᄃᆡ<end_of_turn>
<start_of_turn>model
The lyrics describe a night filled with a mysterious atmosphere and a sense of longing. The speaker wanders through a dark and deserted room, listening to the sounds of the night. The sounds seem to be coming from the room itself, creating an eerie and unsettling atmosphere. The speaker's thoughts are filled with a sen

# LoRA Fine-tuning

In [ ]:
# Enable LoRA for the model and set the LoRA rank (4, 8 or 16).
gemma.backbone.enable_lora(rank=lora_rank)
gemma.summary()

# Limit the input sequence length to 128 (to control memory usage).
gemma.preprocessor.sequence_length = 2048
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=lr_value,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,508,900,352 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,508,900,352 (9.35 GB)

 Trainable params: 2,727,936 (10.41 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

## Save LoRA for each epoch

In [ ]:
for x in range(20):
  gemma.fit(train, epochs=1, batch_size=1)
  model_name = f"/content/drive/MyDrive/{lora_name}_{lora_rank}_epoch{x+1}.lora.h5"
  gemma.backbone.save_lora_weights(model_name)
  text_gen("ᄃᆡ작ᄒᆞ여그ᄭᅩᆺ치흣터지거ᄂᆞᆯ")
  text_gen("금두겁이품의드러뵈니일졍ᄌᆡᄌᆞᄅᆞᆯ나흐리로다ᄒᆞ더니과연그ᄃᆞᆯ부터잉ᄐᆡᄒᆞ여십삭이차니")
  text_gen("이ᄯᆡᄂᆞᆫᄉᆞ월초팔일이라이날밤의오ᄉᆡᆨ구룸이집을두루고향ᄂᆡ진동ᄒᆞ며션녀ᄒᆞᆫᄡᅣᆼ이촉을들고드러와김ᄉᆡᆼᄃᆞ려니르ᄃᆡ")
  text_gen("ᄌᆡ히길너텬졍을어긔지말으소셔이아희ᄇᆡ필은낙양니샹셔집아ᄌᆡ니")

447/447 ━━━━━━━━━━━━━━━━━━━━ 158s 247ms/step - loss: 1.1020 - sparse_categorical_accuracy: 0.3199

Gemma output:
<start_of_turn>user
ᄃᆡ작ᄒᆞ여그ᄭᅩᆺ치흣터지거ᄂᆞᆯ<end_of_turn>
<start_of_turn>model
다작하여 그 밖에 지거늘을 잃어 떨어나라.
TOTAL TIME ELAPSED: 15.38s

Gemma output:
<start_of_turn>user
금두겁이품의드러뵈니일졍ᄌᆡᄌᆞᄅᆞᆯ나흐리로다ᄒᆞ더니과연그ᄃᆞᆯ부터잉ᄐᆡᄒᆞ여십삭이차니<end_of_turn>
<start_of_turn>model
금두겁이 품의 대러 보니 일정한 자사를 나오더니, 과연 그 대를부터 잉타하여십사이 차니
TOTAL TIME ELAPSED: 0.43s

Gemma output:
<start_of_turn>user
이ᄯᆡᄂᆞᆫᄉᆞ월초팔일이라이날밤의오ᄉᆡᆨ구룸이집을두루고향ᄂᆡ진동ᄒᆞ며션녀ᄒᆞᆫᄡᅣᆼ이촉을들고드러와김ᄉᆡᆼᄃᆞ려니르ᄃᆡ<end_of_turn>
<start_of_turn>model
이 침내한 사월 초팔일이 라이날밤에 오사기구룸이 집을 두루고 향내 진동하며 섬녀한 침을 들고 드러와 김사랑 대려니 르 reputa.
TOTAL TIME ELAPSED: 0.60s

Gemma output:
<start_of_turn>user
ᄌᆡ히길너텬졍을어긔지말으소셔이아희ᄇᆡ필은낙양니샹셔집아ᄌᆡ니<end_of_turn>
<start_of_turn>model
자히 길을 떠나 정을 어기 말하시어 이 아희 백필은 낙양니, 샹서 집어 자니
TOTAL TIME ELAPSED: 0.39s
447/447 ━━━━━━━━━━━━━━━━━━━━ 111s 247ms/step - loss: 0.7182 - sparse_categorical_accuracy: 0.4552

Gemma output:
<start_of_turn>user
ᄃᆡ작ᄒᆞ여그ᄭᅩᆺ치흣터지거ᄂᆞᆯ<end_of_turn>
